In [ ]:
pip install transformers

     |████████████████████████████████| 2.3MB 4.0MB/s 
     |████████████████████████████████| 3.3MB 31.3MB/s 
     |████████████████████████████████| 901kB 34.8MB/s 


In [ ]:
import pandas as pd
from sklearn.utils import shuffle

In [ ]:
#New data
#df = pd.read_csv('/content/drive/MyDrive/BERT/Finetuning BERT on EMSCAD/combined_annotations.csv',sep=';')

#Conference data
df = pd.read_csv('/content/drive/MyDrive/BERT/Finetuning BERT on EMSCAD/Conference/conference_data.csv',sep=',')

df = df.fillna('')
#df.head()
df['text'] = df['left_context'] + ' '+df['recovered_gram'] + ' '+df['right_context']
df = df[['text','label']]
df = shuffle(df)

In [ ]:
train_text = df['text'].to_list()
train_labels = df['label'].to_list()
#train_text_sample = train_text[0:16668]
#train_labels_sample = train_labels[0:16668]
#test_texts = train_text[16668:len(train_text)]
#test_labels = train_labels[16668:len(train_text)]

#Conference
train_text_sample = train_text[0:6924]
train_labels_sample = train_labels[0:6924]
test_texts = train_text[6924:len(train_text)]
test_labels = train_labels[6924:len(train_text)]

In [ ]:
from sklearn.model_selection import train_test_split
train_texts, val_texts, train_labels, val_labels = train_test_split(train_text_sample, train_labels_sample, test_size=.2)

In [ ]:
from transformers import BertTokenizerFast
tokenizer = BertTokenizerFast.from_pretrained('bert-base-uncased',num_labels=3)

In [ ]:
train_encodings = tokenizer(train_texts, truncation=True, padding=True)
val_encodings = tokenizer(val_texts, truncation=True, padding=True)
test_encodings = tokenizer(test_texts, truncation=True, padding=True)

In [ ]:
import tensorflow as tf

train_dataset = tf.data.Dataset.from_tensor_slices((
    dict(train_encodings),
    train_labels
))
val_dataset = tf.data.Dataset.from_tensor_slices((
    dict(val_encodings),
    val_labels
))
test_dataset = tf.data.Dataset.from_tensor_slices((
    dict(test_encodings),
    test_labels
))

In [ ]:
tf.test.is_gpu_available()

Instructions for updating:
Use `tf.config.list_physical_devices('GPU')` instead.


True

In [ ]:
from transformers import TFBertForSequenceClassification, BertForSequenceClassification

model = TFBertForSequenceClassification.from_pretrained('bert-base-uncased',num_labels=3)

optimizer = tf.keras.optimizers.Adam(learning_rate=5e-5)
model.compile(optimizer=optimizer, loss=model.compute_loss) # can also use any keras loss fn
#model.fit(train_dataset.shuffle(1000).batch(16), epochs=3, batch_size=16)

All model checkpoint layers were used when initializing TFBertForSequenceClassification.

Some layers of TFBertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
model.fit(train_dataset.shuffle(1000).batch(16), epochs=3, batch_size=16)

Epoch 1/3
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: module, class, method, function, traceback, frame, or code object was expected, got cython_function_or_method
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: module, class, method, function, traceback, frame, or code object was expected, got cython_function_or_method
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Cause: while/else statement not yet supported
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Cause: while/else statement not yet supported
To silence this warning, decorate the function with @tf.autogr

In [ ]:
model.save_pretrained('/content/drive/MyDrive/BERT/Finetuning BERT on EMSCAD/Finetuned model',
                       push_to_hub=True,
                       repo_url='https://huggingface.co/Ivo/emscad-skill-extraction-conference',
                       use_auth_token='uHuXixshJobFWRxPgsjJCLYbeYpTKCBWcLJnNlVUwpkeSRILcDlgiTbYrJhyfLDhFLHJzeBwBJNztbnPltCIzWIgLmQVujHdANVmIpiKLfZBvZUjKjCCuWXAkZwOgFgE'
                      )

('/content/drive/MyDrive/BERT/Finetuning BERT on EMSCAD/Finetuned model/tokenizer_config.json',
 '/content/drive/MyDrive/BERT/Finetuning BERT on EMSCAD/Finetuned model/special_tokens_map.json',
 '/content/drive/MyDrive/BERT/Finetuning BERT on EMSCAD/Finetuned model/vocab.txt',
 '/content/drive/MyDrive/BERT/Finetuning BERT on EMSCAD/Finetuned model/added_tokens.json',
 '/content/drive/MyDrive/BERT/Finetuning BERT on EMSCAD/Finetuned model/tokenizer.json')

In [ ]:
!curl -s https://packagecloud.io/install/repositories/github/git-lfs/script.deb.sh | sudo bash

Detected operating system as Ubuntu/bionic.
Checking for curl...
Detected curl...
Checking for gpg...
Detected gpg...
Running apt-get update... done.
Installing apt-transport-https... done.
Installing /etc/apt/sources.list.d/github_git-lfs.list...done.
Importing packagecloud gpg key... done.
Running apt-get update... done.

The repository is setup! You can now install packages.


In [ ]:
from transformers import TFBertForSequenceClassification, BertForSequenceClassification
pt_model = BertForSequenceClassification.from_pretrained("Ivo/emscad-skill-extraction-conference", from_tf=True)
pt_model.save_pretrained('/content/drive/MyDrive/BERT/Finetuning BERT on EMSCAD/Fine tuned model pytoch',
                      push_to_hub=True,
                      repo_url='https://huggingface.co/Ivo/emscad-skill-extraction-conference',
                      use_auth_token='uHuXixshJobFWRxPgsjJCLYbeYpTKCBWcLJnNlVUwpkeSRILcDlgiTbYrJhyfLDhFLHJzeBwBJNztbnPltCIzWIgLmQVujHdANVmIpiKLfZBvZUjKjCCuWXAkZwOgFgE'
                      )

All TF 2.0 model weights were used when initializing BertForSequenceClassification.

All the weights of BertForSequenceClassification were initialized from the TF 2.0 model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use BertForSequenceClassification for predictions without further training.
